In [22]:
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import  root_mean_squared_error, classification_report, accuracy_score
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import numpy as np
import mlflow

In [15]:

file_path = r'C:\Users\jplv0\PycharmProjects\final-exam-pcd2024-autumn\cleaned_data\cleaned_ticket_classification.csv'
df = pd.read_csv(file_path)

In [16]:
# train, test, split
X = df['complaint_what_happened']
y = df['ticket_classification']

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [6]:
import dagshub
import mlflow
dagshub.init(repo_owner='zapatacc', repo_name='final-exam-pcd2024-autumn', mlflow=True)
mlflow.set_experiment('juan-ley-random-forest')


Accessing as juanplv04

Initialized MLflow to track repo "zapatacc/final-exam-pcd2024-autumn"

Repository zapatacc/final-exam-pcd2024-autumn initialized!

<Experiment: artifact_location='mlflow-artifacts:/6698e85967524b0a8c6fb9f009ae4810', creation_time=1732214488506, experiment_id='19', last_update_time=1732214488506, lifecycle_stage='active', name='juan-ley-random-forest', tags={}>

## Random Forest

In [17]:
mlflow.sklearn.autolog()

# Define objective function for Random Forest model
def objective_rf(params):
    with mlflow.start_run(nested=True):
        # Set model tag
        mlflow.set_tag("model_family", "random_forest")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Create the Random Forest model with given parameters
        rf_model = RandomForestClassifier(
            n_estimators=int(params['n_estimators']),
            max_depth=int(params['max_depth']),
            min_samples_split=int(params['min_samples_split']),
            min_samples_leaf=int(params['min_samples_leaf']),
            random_state=42
        )
        
        # Fit the model
        rf_model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = rf_model.predict(X_test)
        
        # Calculate accuracy
        accuracy = accuracy_score(y_test, y_pred)
        
        # Log accuracy metric
        mlflow.log_metric("accuracy", accuracy)
        
        return {'loss': -accuracy, 'status': STATUS_OK}

# Define search space for RandomForestClassifier
search_space_rf = {
    'n_estimators': hp.quniform('n_estimators', 50, 300, 1),
    'max_depth': hp.quniform('max_depth', 5, 30, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 5, 1),
}

# Run hyperparameter optimization
with mlflow.start_run(run_name="Parent Random Forest", nested=True):
    best_params_rf = fmin(
        fn=objective_rf,
        space=search_space_rf,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    
    # Log best parameters
    mlflow.log_params(best_params_rf)

# Evaluate the best model
y_pred = RandomForestClassifier(
    n_estimators=int(best_params_rf['n_estimators']),
    max_depth=int(best_params_rf['max_depth']),
    min_samples_split=int(best_params_rf['min_samples_split']),
    min_samples_leaf=int(best_params_rf['min_samples_leaf']),
    random_state=42
).fit(X_train, y_train).predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)


2024/11/21 16:32:31 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.1, but the installed version is 1.5.2. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.


  0%|                                                                                                                                            | 0/10 [00:00<?, ?trial/s, best loss=?]

2024/11/21 16:32:32 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'

C:\Users\jplv0\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))

2024/11/21 16:33:24 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 798a4f1c812e4e69aa2da021ee850543. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/11/21 16:33:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run defiant-s

 10%|███████████▎                                                                                                     | 1/10 [00:53<07:57, 53.06s/trial, best loss: -0.5286053255997891]

2024/11/21 16:33:25 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'

C:\Users\jplv0\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))

2024/11/21 16:33:44 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 3af87ecacf504980ba4bd79da81d8ada. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/11/21 16:33:45 INFO mlflow.tracking._tracking_service.client: 🏃 View run capriciou

...

2024/11/21 16:38:34 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'

C:\Users\jplv0\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))

2024/11/21 16:39:30 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 29cbc3c138394dd5a2bb7355444e2eea. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/11/21 16:39:31 INFO mlflow.tracking._tracking_service.client: 🏃 View run auspiciou

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [06:59<00:00, 41.99s/trial, best loss: -0.5286053255997891]


2024/11/21 16:39:32 INFO mlflow.tracking._tracking_service.client: 🏃 View run Parent Random Forest at: https://dagshub.com/zapatacc/final-exam-pcd2024-autumn.mlflow/#/experiments/19/runs/6f77d21c1b0d47c69737f72557ac1962.
2024/11/21 16:39:32 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/zapatacc/final-exam-pcd2024-autumn.mlflow/#/experiments/19.
2024/11/21 16:39:32 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '6233c1f2ae8f4a90a9770deef6511f1b', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/11/21 16:39:32 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'
C:\Users\jplv0\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. 

Accuracy: 0.53
Classification Report:
                                                                                                               precision    recall  f1-score   support

                                                        Bank account or service + (CD) Certificate of deposit       0.00      0.00      0.00         3
                                                 Bank account or service + Cashing a check without an account       0.00      0.00      0.00         7
                                                                   Bank account or service + Checking account       1.00      0.00      0.01       217
                                                         Bank account or service + Other bank product/service       0.00      0.00      0.00        55
                                                                    Bank account or service + Savings account       0.00      0.00      0.00        17
                                                    Che

C:\Users\jplv0\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\jplv0\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\jplv0\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="juan-ley-random-forest-model"
)

Ingrese el run_id 060fd68f8e16463fa7b2341d098d974c


Successfully registered model 'juan-ley-random-forest-model'.
2024/11/21 14:46:37 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: juan-ley-random-forest-model, version 1
Created version '1' of model 'juan-ley-random-forest-model'.


In [10]:
MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()


In [11]:
from datetime import datetime
from mlflow import MlflowClient


client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="juan-ley-random-forest-model",
    description="",
)
new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model 
client.set_registered_model_alias(
    name="juan-ley-random-forest-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="juan-ley-random-forest-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=['champion'], creation_timestamp=1732221997686, current_stage='None', description='The model version 1 was transitioned to champion on 2024-11-21 16:14:07.478835', last_updated_timestamp=1732227247792, name='juan-ley-random-forest-model', run_id='060fd68f8e16463fa7b2341d098d974c', run_link='', source='mlflow-artifacts:/6698e85967524b0a8c6fb9f009ae4810/060fd68f8e16463fa7b2341d098d974c/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [18]:
best_params_rf

{'max_depth': 28.0,
 'min_samples_leaf': 3.0,
 'min_samples_split': 10.0,
 'n_estimators': 132.0}

In [20]:
import mlflow.pyfunc

model_name = "juan-ley-random-forest-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_test)

array(['Checking or savings account + Checking account',
       'Mortgage + Conventional home mortgage',
       'Credit card or prepaid card + General-purpose credit card or charge card',
       ..., 'Checking or savings account + Checking account',
       'Checking or savings account + Checking account',
       'Checking or savings account + Checking account'], dtype=object)

## Logistic Regression

In [24]:
import dagshub
import mlflow
dagshub.init(repo_owner='zapatacc', repo_name='final-exam-pcd2024-autumn', mlflow=True)
mlflow.set_experiment('juan-ley-logistic-regression')

Initialized MLflow to track repo "zapatacc/final-exam-pcd2024-autumn"

Repository zapatacc/final-exam-pcd2024-autumn initialized!

2024/11/21 17:06:06 INFO mlflow.tracking.fluent: Experiment with name 'juan-ley-logistic-regression' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/2ba27dd603444cc7af16985d563b264d', creation_time=1732230366992, experiment_id='26', last_update_time=1732230366992, lifecycle_stage='active', name='juan-ley-logistic-regression', tags={}>

In [25]:
def objective_lr(params):
    with mlflow.start_run(nested=True):
        # Set model tag
        mlflow.set_tag("model_family", "logistic_regression")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Create the Logistic Regression model with given parameters
        lr_model = LogisticRegression(
            C=params['C'],
            max_iter=int(params['max_iter']),
            solver=params['solver'],
            random_state=42
        )
        
        # Fit the model
        lr_model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = lr_model.predict(X_test)
        
        # Calculate accuracy
        accuracy = accuracy_score(y_test, y_pred)
        
        # Log accuracy metric
        mlflow.log_metric("accuracy", accuracy)
        
        return {'loss': -accuracy, 'status': STATUS_OK}
# Define search space for LogisticRegression
search_space_lr = {
    'C': hp.loguniform('C', np.log(0.01), np.log(10)),
    'max_iter': hp.quniform('max_iter', 100, 500, 1),
    'solver': hp.choice('solver', ['liblinear', 'lbfgs'])
}

# Run hyperparameter optimization
with mlflow.start_run(run_name="Parent Logistic Regression", nested=True):
    best_params_lr = fmin(
        fn=objective_lr,
        space=search_space_lr,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    
    # Log best parameters
    mlflow.log_params(best_params_lr)

# Evaluate the best model
y_pred = LogisticRegression(
    C=best_params_lr['C'],
    max_iter=int(best_params_lr['max_iter']),
    solver=['liblinear', 'lbfgs'][best_params_lr['solver']],
    random_state=42
).fit(X_train, y_train).predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=1)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)



  0%|                                                                                                                                            | 0/10 [00:00<?, ?trial/s, best loss=?]

2024/11/21 17:06:14 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'

C:\Users\jplv0\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))

2024/11/21 17:06:34 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 1a5d3c8a10f9498a9b8f28c0c59af1c8. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/11/21 17:06:34 INFO mlflow.tracking._tracking_service.client: 🏃 View run amusing-f

 10%|███████████▏                                                                                                    | 1/10 [00:21<03:10, 21.11s/trial, best loss: -0.40311099393619826]

2024/11/21 17:06:35 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'

C:\Users\jplv0\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))

2024/11/21 17:07:09 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 1ffa6b99ba8e4f67af84ad76461f2d74. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/11/21 17:07:10 INFO mlflow.tracking._tracking_service.client: 🏃 View run painted-k

 20%|██████████████████████▌                                                                                          | 2/10 [00:56<03:57, 29.67s/trial, best loss: -0.6016345900342737]

...

 90%|█████████████████████████████████████████████████████████████████████████████████████████████████████▋           | 9/10 [04:16<00:29, 29.57s/trial, best loss: -0.6016345900342737]

2024/11/21 17:10:30 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'

C:\Users\jplv0\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))

2024/11/21 17:10:52 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID ef915e543f524b20a92cccbc0570ca47. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/11/21 17:10:52 INFO mlflow.tracking._tracking_service.client: 🏃 View run unruly-bi

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [04:39<00:00, 27.92s/trial, best loss: -0.6016345900342737]


2024/11/21 17:10:52 INFO mlflow.tracking._tracking_service.client: 🏃 View run Parent Logistic Regression at: https://dagshub.com/zapatacc/final-exam-pcd2024-autumn.mlflow/#/experiments/26/runs/af8ed1a125af4bbfb1125cbf12334b41.
2024/11/21 17:10:52 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/zapatacc/final-exam-pcd2024-autumn.mlflow/#/experiments/26.
2024/11/21 17:10:53 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '0c4636f07a124fb9995c9d93b0f080a0', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/11/21 17:10:53 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'
C:\Users\jplv0\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted sam

Accuracy: 0.60
Classification Report:
                                                                                                               precision    recall  f1-score   support

                                                        Bank account or service + (CD) Certificate of deposit       1.00      0.00      0.00         3
                                                 Bank account or service + Cashing a check without an account       1.00      0.00      0.00         7
                                                                   Bank account or service + Checking account       0.48      0.47      0.48       217
                                                         Bank account or service + Other bank product/service       0.25      0.04      0.06        55
                                                                    Bank account or service + Savings account       1.00      0.06      0.11        17
                                                    Che

In [26]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="juan-ley-logistic-regression-model"
)

Ingrese el run_id 1ffa6b99ba8e4f67af84ad76461f2d74


Successfully registered model 'juan-ley-logistic-regression-model'.
2024/11/21 17:15:10 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: juan-ley-logistic-regression-model, version 1
Created version '1' of model 'juan-ley-logistic-regression-model'.


In [27]:
from datetime import datetime
from mlflow import MlflowClient


client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="juan-ley-logistic-regression-model",
    description="",
)
new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model 
client.set_registered_model_alias(
    name="juan-ley-logistic-regression-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="juan-ley-logistic-regression-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=['champion'], creation_timestamp=1732230910470, current_stage='None', description='The model version 1 was transitioned to champion on 2024-11-21 17:17:34.428144', last_updated_timestamp=1732231054764, name='juan-ley-logistic-regression-model', run_id='1ffa6b99ba8e4f67af84ad76461f2d74', run_link='', source='mlflow-artifacts:/2ba27dd603444cc7af16985d563b264d/1ffa6b99ba8e4f67af84ad76461f2d74/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [35]:
# registro de challenger
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

challenger_model_version = mlflow.register_model(model_uri=run_uri, name="juan-ley-random-forest-model")
client.set_registered_model_alias(name=model_name, alias='challenger', version=challenger_model_version.version)

Ingrese el run_id 798a4f1c812e4e69aa2da021ee850543


Registered model 'juan-ley-random-forest-model' already exists. Creating a new version of this model...
2024/11/21 18:16:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: juan-ley-random-forest-model, version 3
Created version '3' of model 'juan-ley-random-forest-model'.


RestException: RESOURCE_DOES_NOT_EXIST: Response: {'error_code': 'RESOURCE_DOES_NOT_EXIST'}

In [28]:
best_params_lr

{'C': 3.861292160942015, 'max_iter': 474.0, 'solver': 1}

In [30]:
# registro de champion
import mlflow.pyfunc

model_name = "juan-ley-logistic-regression-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_test)

array(['Vehicle loan or lease + Loan',
       'Mortgage + Conventional home mortgage',
       'Credit card or prepaid card + General-purpose credit card or charge card',
       ..., 'Checking or savings account + Checking account',
       'Checking or savings account + Checking account',
       'Checking or savings account + Checking account'], dtype=object)

In [31]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"


Ingrese el run_id 0c4636f07a124fb9995c9d93b0f080a0


In [32]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="juan-ley-logistic-regression-model",
    description="anadir challenger",
)
new_alias = "challenger"
date = datetime.today()
model_version = "1"

# create "challenger" alias for version 1 of model 
client.set_registered_model_alias(
    name="juan-ley-logistic-regression-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="juan-ley-logistic-regression-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=['challenger', 'champion'], creation_timestamp=1732230910470, current_stage='None', description=('The model version 1 was transitioned to challenger on 2024-11-21 '
 '17:32:48.357561'), last_updated_timestamp=1732231968794, name='juan-ley-logistic-regression-model', run_id='1ffa6b99ba8e4f67af84ad76461f2d74', run_link='', source='mlflow-artifacts:/2ba27dd603444cc7af16985d563b264d/1ffa6b99ba8e4f67af84ad76461f2d74/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [33]:
# registro de challenger

challenger_model_version = mlflow.register_model(model_uri=run_uri, name="juan-ley-logistic-regression-model")
client.set_registered_model_alias(name=model_name, alias='challenger', version=challenger_model_version.version)

Registered model 'juan-ley-logistic-regression-model' already exists. Creating a new version of this model...
2024/11/21 17:40:18 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: juan-ley-logistic-regression-model, version 2
Created version '2' of model 'juan-ley-logistic-regression-model'.
